픽셀을 128 x 128일 때 성능을 검증하기 위한 코드입니다.
원래는 정의해 놓은 함수를 불러와서 사용해야 했으나, 검증 과정에서 일부 함수를 직접 코드를 복사해와서 테스트 한 코드도 많습니다.

In [1]:
import cv2
from PIL import Image
import sys
import os
import numpy as np

current_dir = os.getcwd()
sys.path.append(current_dir)
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())
image_folder_path = './G1020/Images/'  

In [2]:
import pandas as pd
df = pd.read_csv('G1020.csv')
image_files = df['imageID'].tolist()  # 이미지 파일 이름이 있는 열 이름
labels = df['binaryLabels'].values  # 레이블이 있는 열 이름

# 이미지 크기 설정
target_size = (28, 28)

# cv2로 이미지 읽고 전처리
def load_and_extract_hue(image_path, target_size):
    image = cv2.imread(image_path)  # 이미지 읽기 (기본 BGR 형식)
    if image is None:
        print(f"Image not found: {image_path}")
        return None
    image = cv2.resize(image, target_size)  # 이미지 크기 조정
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)  # BGR을 HSV로 변환
    hue_channel = hsv_image[:, :, 0]  # Hue 채널만 추출
    hue_channel = hue_channel / 180.0  # Hue 값 정규화 (0~1 범위로 스케일링, Hue 범위는 0-179)
    return hue_channel

# 모든 이미지를 불러와서 리스트에 저장
images = [load_and_extract_hue(image_folder_path + img_path, target_size) for img_path in image_files]
images = np.array([img for img in images if img is not None])  # None 값 제거

# 차원 추가하여 (1020, 1, 128, 128) 형태로 변환
images = images[:, np.newaxis, :, :]

print(f"Processed Hue images shape: {images.shape}")
print(f"Labels shape: {labels.shape}")

Processed Hue images shape: (1020, 1, 28, 28)
Labels shape: (1020,)


In [4]:
from mymethod.neural_network import *
from common.trainer import Trainer
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


max_epochs = 20

network = ConvNet3Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:2.299114715745361
=== epoch:1, train acc:0.3, test acc:0.35 ===
train loss:2.2952759389177113
train loss:2.2872113767696556
train loss:2.2757070889892246
train loss:2.256803138366368
train loss:2.224379310110783
train loss:2.186794958498419
train loss:2.1343941470789174
train loss:2.055350904231305
train loss:1.9875604510778544
train loss:1.8664347012911144
train loss:1.7323561006615662
train loss:1.5814499769928
train loss:1.4086798400710738
train loss:1.2455491701441679
train loss:1.0427143720833054
train loss:0.9450021261133363
train loss:0.6965478797147304
train loss:0.7101846967663834
train loss:0.6755809507796537
train loss:0.6651283876848142
train loss:0.513361261729715
train loss:0.6655281194037348
train loss:0.8487836640284534
train loss:0.830360665558435
train loss:0.32733835191957367
train loss:0.3313824765267569
train loss:0.6869040825618504
train loss:0.8968981181445157
train loss:0.3612209699890836
train loss:0.6285578790533477
train loss:0.5304414998472097
tra

In [8]:
max_epochs = 20

network = DeepConvNet()
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:2.0262690777760453
=== epoch:1, train acc:0.7, test acc:0.63 ===
train loss:2.2404881357291053
train loss:2.2015569707436873
train loss:1.0344168206949351
train loss:2.039315841224476
train loss:1.7942754423420535
train loss:2.2715692297677004
train loss:1.7332602523752136
train loss:1.8409789384577884
train loss:2.145710456817768
train loss:1.5033178010315893
train loss:0.9963202274157072
train loss:1.9589717212554336
train loss:1.5360352247413818
train loss:1.8408652696529295
train loss:1.288656168239756
train loss:1.4911821528330387
train loss:1.377948420091246
train loss:1.6725964312862736
train loss:2.0937796514885942
train loss:1.6056728478574183
train loss:2.1782459406937797
train loss:1.2814115925846095
train loss:1.7331143333321013
train loss:1.843718259594624
train loss:1.4565325049523508
train loss:1.9646825134432384
train loss:2.0432454242580986
train loss:0.9798496749767966
train loss:1.7561495437797046
train loss:2.026430854368071
train loss:1.6972630685055328


In [9]:
import pandas as pd
df = pd.read_csv('G1020.csv')
image_files = df['imageID'].tolist()  # 이미지 파일 이름이 있는 열 이름
labels = df['binaryLabels'].values  # 레이블이 있는 열 이름

# 이미지 크기 설정
target_size = (128, 128)

# cv2로 이미지 읽고 전처리
def load_and_extract_hue(image_path, target_size):
    image = cv2.imread(image_path)  # 이미지 읽기 (기본 BGR 형식)
    if image is None:
        print(f"Image not found: {image_path}")
        return None
    image = cv2.resize(image, target_size)  # 이미지 크기 조정
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)  # BGR을 HSV로 변환
    hue_channel = hsv_image[:, :, 0]  # Hue 채널만 추출
    hue_channel = hue_channel / 180.0  # Hue 값 정규화 (0~1 범위로 스케일링, Hue 범위는 0-179)
    return hue_channel

# 모든 이미지를 불러와서 리스트에 저장
images = [load_and_extract_hue(image_folder_path + img_path, target_size) for img_path in image_files]
images = np.array([img for img in images if img is not None])  # None 값 제거

# 차원 추가하여 (1020, 1, 128, 128) 형태로 변환
images = images[:, np.newaxis, :, :]

print(f"Processed Hue images shape: {images.shape}")
print(f"Labels shape: {labels.shape}")

Processed Hue images shape: (1020, 1, 128, 128)
Labels shape: (1020,)


In [10]:
class ConvNet3Layer:
    def __init__(self, input_dim=(1, 128, 128), 
                 conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))

        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)


        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]

    def numerical_gradient(self, x, t):
        loss_w = lambda w: self.loss(x, t)

        grads = {}
        for idx in (1, 2, 3):
            grads['W' + str(idx)] = numerical_gradient(loss_w, self.params['W' + str(idx)])
            grads['b' + str(idx)] = numerical_gradient(loss_w, self.params['b' + str(idx)])

        return grads

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

In [13]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


max_epochs = 20

network = ConvNet3Layer(input_dim=(1,128,128), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:2.263087000663222
=== epoch:1, train acc:0.3, test acc:0.35 ===
train loss:2.148714996301905
train loss:1.9701040603254267
train loss:1.6794971416648246
train loss:1.2350760931288882
train loss:0.8739562522984807
train loss:1.0262891402369605
train loss:0.5296248527304634
train loss:0.6698278664790294
train loss:0.5340600293692731
train loss:0.6182808194116785
train loss:0.5859891323992127
train loss:0.49873431883283115
train loss:0.7067004682783964
train loss:0.6480311971603624
train loss:0.6870581085078735
train loss:0.6150200085548787
train loss:0.7231432170658286
train loss:0.5894746201366138
train loss:0.6198315559182638
train loss:0.6883161421894581
train loss:0.6886586813602908
train loss:0.7146126021321451
train loss:0.6405563352330242
train loss:0.746649834033072
train loss:0.4668000058317573
train loss:0.30256315607433837
train loss:0.6254141740006554
train loss:0.6627269710366838
train loss:0.736916152095501
train loss:0.6079634613533484
train loss:0.4825234069583

In [16]:
max_epochs = 20

network = ConvNet5Layer(input_dim=(1,128,128), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

ValueError: operands could not be broadcast together with shapes (10,100) (10,) 

In [38]:
class DeepConvNet:
    """정확도 99% 이상의 고정밀 합성곱 신경망

    네트워크 구성은 아래와 같음
        conv - relu - conv- relu - pool -
        conv - relu - conv- relu - pool -
        conv - relu - conv- relu - pool -
        affine - relu - dropout - affine - dropout - softmax
    """
    def __init__(self, input_dim=(1, 128, 128),
                 conv_param_1 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_2 = {'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_3 = {'filter_num':32, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_4 = {'filter_num':32, 'filter_size':3, 'pad':2, 'stride':1},
                 conv_param_5 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_6 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 hidden_size=50, output_size=10):
        # 가중치 초기화===========
        # 각 층의 뉴런 하나당 앞 층의 몇 개 뉴런과 연결되는가（TODO: 자동 계산되게 바꿀 것）
        pre_node_nums = np.array([1*3*3, 16*3*3, 16*3*3, 32*3*3, 32*3*3, 64*3*3, 64*4*4, hidden_size])
        wight_init_scales = np.sqrt(2.0 / pre_node_nums)  # ReLU를 사용할 때의 권장 초깃값
        
        self.params = {}
        pre_channel_num = input_dim[0]
        for idx, conv_param in enumerate([conv_param_1, conv_param_2, conv_param_3, conv_param_4, conv_param_5, conv_param_6]):
            self.params['W' + str(idx+1)] = wight_init_scales[idx] * np.random.randn(conv_param['filter_num'], pre_channel_num, conv_param['filter_size'], conv_param['filter_size'])
            self.params['b' + str(idx+1)] = np.zeros(conv_param['filter_num'])
            pre_channel_num = conv_param['filter_num']
        self.params['W7'] = wight_init_scales[6] * np.random.randn(64*16*16, hidden_size)
        self.params['b7'] = np.zeros(hidden_size)
        self.params['W8'] = wight_init_scales[7] * np.random.randn(hidden_size, output_size)
        self.params['b8'] = np.zeros(output_size)

        # 계층 생성===========
        self.layers = []
        self.layers.append(Convolution(self.params['W1'], self.params['b1'], 
                           conv_param_1['stride'], conv_param_1['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W2'], self.params['b2'], 
                           conv_param_2['stride'], conv_param_2['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W3'], self.params['b3'], 
                           conv_param_3['stride'], conv_param_3['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W4'], self.params['b4'],
                           conv_param_4['stride'], conv_param_4['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W5'], self.params['b5'],
                           conv_param_5['stride'], conv_param_5['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W6'], self.params['b6'],
                           conv_param_6['stride'], conv_param_6['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Affine(self.params['W7'], self.params['b7']))
        self.layers.append(Relu())
        self.layers.append(Dropout(0.5))
        self.layers.append(Affine(self.params['W8'], self.params['b8']))
        self.layers.append(Dropout(0.5))
        
        self.last_layer = SoftmaxWithLoss()

    def predict(self, x, train_flg=False):
        for layer in self.layers:
            if isinstance(layer, Dropout):
                x = layer.forward(x, train_flg)
            else:
                x = layer.forward(x)
        return x

    def loss(self, x, t):
        y = self.predict(x, train_flg=True)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        acc = 0.0

        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx, train_flg=False)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt)

        return acc / x.shape[0]

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        tmp_layers = self.layers.copy()
        tmp_layers.reverse()
        for layer in tmp_layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            grads['W' + str(i+1)] = self.layers[layer_idx].dW
            grads['b' + str(i+1)] = self.layers[layer_idx].db

        return grads

    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            self.layers[layer_idx].W = self.params['W' + str(i+1)]
            self.layers[layer_idx].b = self.params['b' + str(i+1)]

In [39]:
max_epochs = 20

network = DeepConvNet()
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:1.9709256806202604
=== epoch:1, train acc:0.7, test acc:0.65 ===
train loss:1.3984322394348672
train loss:1.8187798311885401
train loss:2.116549593548173
train loss:1.7508551644139583
train loss:2.3349759407753137
train loss:1.94194094006544
train loss:1.2858997999051651
train loss:1.4445398531766183
train loss:2.2821732060023057
train loss:2.132841022526148
train loss:1.5270044188065321
train loss:1.8285390357100517
train loss:0.8955183114415768
train loss:1.820848480323417
train loss:1.627589939774459
train loss:1.469362623766903
train loss:1.5117115772516214
train loss:1.3978751131508467
train loss:1.6771010936156696
train loss:1.9361185380907777
train loss:1.8035171376436232
train loss:2.2461845649977845
train loss:1.3021557875535663
train loss:1.945649166845417
train loss:1.190166288478663
train loss:1.397574496770248
train loss:1.8441630104513405
train loss:1.5414115118522769
train loss:1.0834246053963557
train loss:1.2237589262792052
train loss:1.5768207500832456
trai

In [43]:
from mymethod.functions import *
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.datasets import make_classification


x_train_reshaped = x_train.reshape(x_train.shape[0], -1)
x_test_reshaped = x_test.reshape(x_test.shape[0], -1)
model = LogisticRegression()
model.fit(x_train_reshaped, y_train)
y_pred = model.predict(x_test_reshaped)


accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("정확도:", accuracy)
print("혼동 행렬:\n", conf_matrix)
print("분류 보고서:\n", class_report)

정확도: 0.3333333333333333
혼동 행렬:
 [[  2 136]
 [  0  66]]
분류 보고서:
               precision    recall  f1-score   support

           0       1.00      0.01      0.03       138
           1       0.33      1.00      0.49        66

    accuracy                           0.33       204
   macro avg       0.66      0.51      0.26       204
weighted avg       0.78      0.33      0.18       204



In [44]:
x_train_reshaped = x_train.reshape(x_train.shape[0], -1)
x_test_reshaped = x_test.reshape(x_test.shape[0], -1)
model = LinearDiscriminantAnalysis()
model.fit(x_train_reshaped, y_train)
y_pred = model.predict(x_test_reshaped)


accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("정확도:", accuracy)
print("혼동 행렬:\n", conf_matrix)
print("분류 보고서:\n", class_report)

정확도: 0.5980392156862745
혼동 행렬:
 [[96 42]
 [40 26]]
분류 보고서:
               precision    recall  f1-score   support

           0       0.71      0.70      0.70       138
           1       0.38      0.39      0.39        66

    accuracy                           0.60       204
   macro avg       0.54      0.54      0.54       204
weighted avg       0.60      0.60      0.60       204



In [45]:
x_train_reshaped = x_train.reshape(x_train.shape[0], -1)
x_test_reshaped = x_test.reshape(x_test.shape[0], -1)
model = GaussianNaiveBayes()
model.fit(x_train_reshaped, y_train)
y_pred = model.predict(x_test_reshaped)


accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("정확도:", accuracy)
print("혼동 행렬:\n", conf_matrix)
print("분류 보고서:\n", class_report)

정확도: 0.6764705882352942
혼동 행렬:
 [[138   0]
 [ 66   0]]
분류 보고서:
               precision    recall  f1-score   support

           0       0.68      1.00      0.81       138
           1       0.00      0.00      0.00        66

    accuracy                           0.68       204
   macro avg       0.34      0.50      0.40       204
weighted avg       0.46      0.68      0.55       204



C:\Users\hotmi\OneDrive\Desktop\4학년 2학기\딥러닝\과제\mymethod\functions.py:87: RuntimeWarning: divide by zero encountered in log
  class_conditional = -0.5 * np.sum(np.log(2. * np.pi * self.variances[cls])) - 0.5 * np.sum(((x - self.means[cls]) ** 2) / (self.variances[cls]))
C:\Users\hotmi\OneDrive\Desktop\4학년 2학기\딥러닝\과제\mymethod\functions.py:87: RuntimeWarning: invalid value encountered in divide
  class_conditional = -0.5 * np.sum(np.log(2. * np.pi * self.variances[cls])) - 0.5 * np.sum(((x - self.means[cls]) ** 2) / (self.variances[cls]))
C:\Users\hotmi\OneDrive\Desktop\4학년 2학기\딥러닝\과제\mymethod\functions.py:87: RuntimeWarning: divide by zero encountered in divide
  class_conditional = -0.5 * np.sum(np.log(2. * np.pi * self.variances[cls])) - 0.5 * np.sum(((x - self.means[cls]) ** 2) / (self.variances[cls]))
C:\Users\hotmi\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted sam

In [52]:
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB


x_train_reshaped = x_train.reshape(x_train.shape[0], -1)
x_test_reshaped = x_test.reshape(x_test.shape[0], -1)

# Logistic Regression model
logistic_model = LogisticRegression()
logistic_model.fit(x_train_reshaped, y_train)
y_pred_logistic = logistic_model.predict(x_test_reshaped)
logistic_accuracy = accuracy_score(y_test, y_pred_logistic)
print(f"Logistic Regression Accuracy: {logistic_accuracy:.4f}")

# Linear Discriminant Analysis (LDA) model
lda_model = LinearDiscriminantAnalysis()
lda_model.fit(x_train_reshaped, y_train)
y_pred_lda = lda_model.predict(x_test_reshaped)
lda_accuracy = accuracy_score(y_test, y_pred_lda)
print(f"LDA Accuracy: {lda_accuracy:.4f}")

# Gaussian Naive Bayes classifier
gaussian_model = GaussianNB()
gaussian_model.fit(x_train_reshaped, y_train)
y_pred_gaussian = gaussian_model.predict(x_test_reshaped)
gaussian_accuracy = accuracy_score(y_test, y_pred_gaussian)
print(f"Gaussian Classifier Accuracy: {gaussian_accuracy:.4f}")


Logistic Regression Accuracy: 0.6225
LDA Accuracy: 0.6618
Gaussian Classifier Accuracy: 0.4167


In [53]:
import tensorflow as tf
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import numpy as np



# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(x_train_reshaped )
X_test = scaler.transform(x_test_reshaped)

# Build a simple TensorFlow model for binary classification
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model on the test set
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()
accuracy = accuracy_score(y_test, y_pred)
print(f"TensorFlow Model Accuracy: {accuracy:.4f}")

Epoch 1/20


C:\Users\hotmi\anaconda3\Lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.6018 - loss: 1.5149 - val_accuracy: 0.5122 - val_loss: 1.2399
Epoch 2/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7167 - loss: 0.7358 - val_accuracy: 0.7256 - val_loss: 0.8204
Epoch 3/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8376 - loss: 0.3822 - val_accuracy: 0.6341 - val_loss: 0.9786
Epoch 4/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8567 - loss: 0.3852 - val_accuracy: 0.5915 - val_loss: 0.9949
Epoch 5/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8611 - loss: 0.3844 - val_accuracy: 0.6341 - val_loss: 0.9125
Epoch 6/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9367 - loss: 0.2319 - val_accuracy: 0.6829 - val_loss: 0.9385
Epoch 7/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9647 - loss: 0.1694 - val_accuracy: 0.6768 - val_loss: 0.9957
Epoch 8/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9768 - loss: 0.1170 - val_accuracy: 0.7012 - val_loss: 1.0864
Ep

In [50]:
# Build a deeper TensorFlow model for binary classification with CNN layers
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train.shape[1], 1)),
    tf.keras.layers.Reshape((X_train.shape[1], 1)),
    tf.keras.layers.Conv1D(32, kernel_size=3, activation='relu'),
    tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(128, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model on the test set
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()
accuracy = accuracy_score(y_test, y_pred)
print(f"TensorFlow Model Accuracy: {accuracy:.4f}")


C:\Users\hotmi\anaconda3\Lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step - accuracy: 0.6180 - loss: 2.1614 - val_accuracy: 0.6951 - val_loss: 0.6749
Epoch 2/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.7053 - loss: 0.6262 - val_accuracy: 0.6951 - val_loss: 0.6505
Epoch 3/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 43s 2s/step - accuracy: 0.7342 - loss: 0.5569 - val_accuracy: 0.6829 - val_loss: 0.6285
Epoch 4/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.7635 - loss: 0.5049 - val_accuracy: 0.6890 - val_loss: 0.6681
Epoch 5/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.7958 - loss: 0.4496 - val_accuracy: 0.7073 - val_loss: 0.6817
Epoch 6/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.8948 - loss: 0.3280 - val_accuracy: 0.6890 - val_loss: 0.7100
Epoch 7/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.9486 - loss: 0.2351 - val_accuracy: 0.6402 - val_loss: 0.9414
Epoch 8/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.9831 - loss: 0.0874 - val_accuracy: 0.6768 - val_loss:

In [51]:
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()
accuracy = accuracy_score(y_test, y_pred)
print(f"TensorFlow Model Accuracy: {accuracy:.4f}")


7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 212ms/step
TensorFlow Model Accuracy: 0.6520


In [54]:
class ConvNet3Layer:
    def __init__(self, input_dim=(1, 28, 28), 
                 conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))

        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)


        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]

    def numerical_gradient(self, x, t):
        loss_w = lambda w: self.loss(x, t)

        grads = {}
        for idx in (1, 2, 3):
            grads['W' + str(idx)] = numerical_gradient(loss_w, self.params['W' + str(idx)])
            grads['b' + str(idx)] = numerical_gradient(loss_w, self.params['b' + str(idx)])

        return grads

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads
        

In [55]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


max_epochs = 20

network = ConvNet3Layer(input_dim=(1,128,128), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:2.257426039307062
=== epoch:1, train acc:0.7, test acc:0.65 ===
train loss:2.125987266181375
train loss:1.9140892729886096
train loss:1.6465471107620977
train loss:1.2182149193305463
train loss:0.7561317569931127
train loss:0.6494452691191397
train loss:0.3185993602909669
train loss:0.7839215184733372
train loss:0.6640310813006123
train loss:0.6191931950734172
train loss:0.7018489944556301
train loss:0.7169834316683076
train loss:0.6158929755513463
train loss:0.7341478276676666
train loss:0.46689367882428395
train loss:0.31916244772681585
train loss:1.0542891742719567
train loss:0.6676881008335608
train loss:0.6938733897954155
train loss:0.874185316581882
train loss:0.9207606929821551
train loss:0.7652238697430231
train loss:0.6705939235926883
train loss:0.35420071287070487
train loss:0.9157949831653678
train loss:0.9132488553334198
train loss:0.7877228972928596
train loss:0.8779605339484778
train loss:0.8726320690004755
train loss:0.4519996007485122
train loss:0.65724115960

KeyboardInterrupt: 

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


max_epochs = 20

network = ConvNet3Layer(input_dim=(1,128,128), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=200, output_size=10, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


max_epochs = 20

network = ConvNet3Layer(input_dim=(1,128,128), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=400, output_size=10, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


max_epochs = 20

network = ConvNet3Layer(input_dim=(1,128,128), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=800, output_size=10, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()